In [1]:
import osmnx as ox
import pandas as pd
import pickle

In [2]:
# read the sensor locations from the CSV file
df = pd.read_csv('denhaag_locations_type.csv')

In [3]:
# get the OpenStreetMap road network for the city of The Hague
city = 'The Hague, Netherlands'
G = ox.graph_from_place(city, network_type='drive')

In [4]:
# compute the distances between all pairs of sensors
distances = []
for i in range(len(df)):
    
    lat1, lon1, from_code, from_type, from_dir = df.loc[i, ['latitude', 'longitude', 'code', 'type', 'direction']]
    orig_node = ox.distance.nearest_nodes(G, lon1, lat1)
    
    for j in range(i, len(df)):
        lat2, lon2, to_code, to_type, to_dir = df.loc[j, ['latitude', 'longitude', 'code', 'type', 'direction']]
        dest_node = ox.distance.nearest_nodes(G, lon2, lat2)
        
        route = ox.shortest_path(G, orig_node, dest_node, weight='length') 
        distance = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length')) 
        distances.append([df.loc[i, 'sensor_id'], df.loc[j, 'sensor_id'], from_code, to_code, from_type, to_type, from_dir, to_dir ,distance])
        
        if i != j:
        
            route = ox.shortest_path(G, dest_node, orig_node, weight='length')
            distance = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))
            distances.append([df.loc[j, 'sensor_id'], df.loc[i, 'sensor_id'], to_code, from_code, to_type, from_type, to_dir, from_dir ,distance])

In [5]:
# save into a dataframe and round the distances to 1 decimal
df_distances = pd.DataFrame(distances, columns=['from', 'to', 'from_code', 'to_code', 'from_type', 'to_type', 'from_dir', 'to_dir', 'distance' ])
df_distances['distance'] = df_distances['distance'].round(1)

In [6]:

# pickle the dataframe
with open('all_distances.pickle', 'wb') as f:
    pickle.dump(df_distances, f)

# save the dataframe to a CSV file
df_distances.to_csv('denhaag_all_distances.csv', index=False)
